In [217]:
import pandas as pd
import glob
import json
import datetime
import dateutil
import statistics

## Preprocessing Classes

### Heart Preprocessing

In [384]:
class PreProcess_Heart:

  def MergeHeartData(path):
    #storedDataPAth = '../merged/'
    newPath = 'merged/'
    if not os.path.exists(newPath):
      merged_path = os.mkdir(newPath)
    else:
      merged_path = newPath


    merged_files = list()
    userFolders = sorted([userFolder for userFolder in glob.glob(path + '/**') if not userFolder.startswith('.') and 'merged' not in userFolder])
    users = [person for person in os.listdir(path)]
    feature = 'heart'
    #print(userFolders)
    for user in users:
      data_folder = user + '/'
      if not os.path.exists(merged_path + data_folder):
        os.mkdir(merged_path + data_folder)
      else:
        data_path = merged_path + data_folder
      result = list()
      files = [feature_file for feature_file in glob.glob(path + user + '/' + feature + '/*.json')]
      #print(files)
      for file in files:
        #print('')
        with open(file, "r") as infile:
            
          result.append(json.load(infile))
            #print(result)
          
      #print(result)
      #print(merged_path)
      with open(merged_path + data_folder +  feature  + ".json", "w") as merged_file:
        #print(merged_file)
        json.dump(result, merged_file)
        #print(type(result[0]))
       
          

  def ReOrder(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.')])
    #print(mergedUsers)
    
    for mergedUser in mergedUsers:
      files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'heart' in feature_file]
      #print(files)
      for file in files:
        with open(file, "r") as unsorted_file:
          json_file = json.load(unsorted_file)
          if 'heart' in file:
            sorted_file = sorted(json_file, key = lambda i: i['activities-heart'][0]['dateTime'])
        with open(file, "w") as unsorted_file:
          json.dump(sorted_file, unsorted_file)

  #def DeleteDuplicates():
  # # Re-Index
  def toDataFrame(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.')])
    hearts = []
    print(mergedUsers)
    for mergedUser in mergedUsers:
        files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'heart' in feature_file]
        for file in files:
            heart_dicts = []
            with open(file, "r") as raw_file:
                json_file = json.load(raw_file)
                if 'heart' in file:
                    heart_dicts = [pd.DataFrame.from_dict(pd.json_normalize(json_file[i]['activities-heart-intraday']['dataset'])) for i in range(len(json_file)) if len(json_file[i]['activities-heart-intraday']['dataset']) > 0]
                    #print(len(heart_dicts), len(json_file))
                    for i in range(len(heart_dicts)):
                      date = json_file[i]['activities-heart'][0]['dateTime']
                      heart_dicts[i] = heart_dicts[i].assign(time = lambda x: (date + ' ' + x['time']))
                      heart_dicts[i].rename(columns = {'time':'dateTime'}, inplace = True)
                    hearts.append(heart_dicts)
    return hearts

  def extractData(data, startTime, endTime):
    #pandas.to_datetime
    users = []
    for user in data:
      for i in range(len(user)):
        user[i] = user[i].assign(dateTime = lambda x: (pd.to_datetime(user[i]['dateTime'], format="%Y-%m-%d %H:%M:%S")))

    data_windows = []
    for user in range(len(data)) :
        participant_windows = []
        for day in data[user]:
          half = day[day['dateTime'] >= pd.to_datetime(startTime, format="%Y-%m-%d %H:%M:%S")]
          final = half[half['dateTime'] <= pd.to_datetime(endTime, format="%Y-%m-%d %H:%M:%S")]
          participant_windows.append(final)
        data_windows.append(participant_windows)
    return data_windows


### Step PrePRocessing

In [348]:
class PreProcess_Steps:
  def MergeStepData(path):
    #storedDataPAth = '../merged/'
    newPath = 'merged/'
    if not os.path.exists(newPath):
      merged_path = os.mkdir(newPath)
    else:
      merged_path = newPath


    merged_files = list()
    userFolders = sorted([userFolder for userFolder in glob.glob(path + '/**') if not userFolder.startswith('.') and 'merged' not in userFolder])
    users = [person for person in os.listdir(path)]
    feature = 'steps'
    #print(userFolders)
    for user in users:
      data_folder = user + '/'
      if not os.path.exists(merged_path + data_folder):
        os.mkdir(merged_path + data_folder)
      else:
        data_path = merged_path + data_folder
      result = list()
      files = [feature_file for feature_file in glob.glob(path + user + '/' + feature + '/*.json')]
      #print(files)
      for file in files:
        #print('')
        with open(file, "r") as infile:
            
          result.append(json.load(infile))
            #print(result)
          
      #print(result)
      #print(merged_path)
      with open(merged_path + data_folder +  feature  + ".json", "w") as merged_file:
        #print(merged_file)
        json.dump(result, merged_file)
   
  def ReOrder(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.') and 'merged' in mergedUser])
    
    for mergedUser in mergedUsers:
      files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'step' in feature_file]
      for file in files:
        with open(file, "r") as unsorted_file:
          json_file = json.load(unsorted_file)
          if 'steps' in file:
            sorted_file = sorted(json_file, key = lambda i: i['activities-steps'][0]['dateTime'])
        with open(file, "w") as unsorted_file:
          json.dump(sorted_file, unsorted_file)

  #def DeleteDuplicates():
  #  Re-Index
  def toDataFrame(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.')])
    steps = []
    #print(mergedUsers)
    for mergedUser in mergedUsers:
        files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'steps' in feature_file]
        for file in files:
            step_dicts = []
            with open(file, "r") as raw_file:
                json_file = json.load(raw_file)
                if 'step' in file:
                    step_dicts = [pd.DataFrame.from_dict(pd.json_normalize(json_file[i]['activities-steps-intraday']['dataset'])) for i in range(len(json_file)) if len(json_file[i]['activities-steps-intraday']['dataset']) > 0]
                    #print(len(heart_dicts), len(json_file))
                    for i in range(len(step_dicts)):
                      date = json_file[i]['activities-steps'][0]['dateTime']
                      step_dicts[i] = step_dicts[i].assign(time = lambda x: (date + ' ' + x['time']))
                      step_dicts[i].rename(columns = {'time':'dateTime'}, inplace = True)
                    steps.append(step_dicts)
    return steps

 def extractData(data, startTime, endTime):
    #pandas.to_datetime
    users = []
    for user in data:
      for i in range(len(user)):
        user[i] = user[i].assign(dateTime = lambda x: (pd.to_datetime(user[i]['dateTime'], format="%Y-%m-%d %H:%M:%S")))

    data_windows = []
    for user in range(len(data)) :
        participant_windows = []
        for day in data[user]:
          half = day[day['dateTime'] >= pd.to_datetime(startTime, format="%Y-%m-%d %H:%M:%S")]
          final = half[half['dateTime'] <= pd.to_datetime(endTime, format="%Y-%m-%d %H:%M:%S")]
          participant_windows.append(final)
        data_windows.append(participant_windows)
    return data_windows

### Sleep PreProcessing

In [349]:
class PreProcess_Sleep:

  def MergeSleepData(path):
    #storedDataPAth = '../merged/'
    newPath = 'merged/'
    if not os.path.exists(newPath):
      merged_path = os.mkdir(newPath)
    else:
      merged_path = newPath


    merged_files = list()
    userFolders = sorted([userFolder for userFolder in glob.glob(path + '/**') if not userFolder.startswith('.') and 'merged' not in userFolder])
    users = [person for person in os.listdir(path)]
    feature = 'sleep'
    #print(userFolders)
    for user in users:
      data_folder = user + '/'
      if not os.path.exists(merged_path + data_folder):
        os.mkdir(merged_path + data_folder)
      else:
        data_path = merged_path + data_folder
      result = list()
      files = [feature_file for feature_file in glob.glob(path + user + '/' + feature + '/*.json')]
      #print(files)
      for file in files:
        #print('')
        with open(file, "r") as infile:
            
          result.append(json.load(infile))
            #print(result)
          
      #print(result)
      #print(merged_path)
      with open(merged_path + data_folder +  feature  + ".json", "w") as merged_file:
        #print(merged_file)
        json.dump(result, merged_file)
          #print(type(result[0]))
       
          

  def ReOrder(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.') and 'merged' in mergedUser])
    
    for mergedUser in mergedUsers:
      files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'sleep' in feature_file]
      for file in files:
        with open(file, "r") as unsorted_file:
          json_file = json.load(unsorted_file)
          if 'sleep' in file:
            json_files = [file for file in json_file if len(file['sleep']) > 0]
            sorted_file = sorted(json_files, key = lambda i: i['sleep'][0]['dateOfSleep'])
        with open(file, "w") as unsorted_file:
          json.dump(sorted_file, unsorted_file)

  #def DeleteDuplicates():
  # # Re-Index
  def toDataFrame(path):
    mergedUsers = sorted([mergedUser for mergedUser in glob.glob('merged/' + '/**') if not mergedUser.startswith('.') and 'merged' in mergedUser])
    sleeps = []
    for mergedUser in mergedUsers:
        files = [feature_file for feature_file in glob.glob(mergedUser + '/*.json') if 'sleep' in feature_file]
        for file in files:
            sleep_dicts = []
            with open(file, "r") as raw_file:
                json_file = json.load(raw_file)
                if 'sleep' in file:
                    sleep_dicts = list()
                    for i in range(len(json_file)):
                    #print(len(json_file[i]['sleep']))
                        dailySleep = json_file[i]['sleep'][0]['minuteData']
                        if len(json_file[i]['sleep']) > 1:
                            j = 1
                            for j in range(len(json_file[i]['sleep'])):
                                dailySleep.extend(json_file[i]['sleep'][j]['minuteData'])
                        sleep_dicts.append(pd.DataFrame.from_dict(pd.json_normalize(sorted(dailySleep, key = lambda x: x['dateTime']))))
                    for i in range(len(sleep_dicts)):
                      date = json_file[i]['sleep'][0]['dateOfSleep']
                      sleep_dicts[i] = sleep_dicts[i].assign(dateTime = lambda x: (date + ' ' + x['dateTime']))
                    sleeps.append(sleep_dicts)
    return sleeps

  def extractData(data, startTime, endTime):
    #pandas.to_datetime
    users = []
    for user in data:
      for i in range(len(user)):
        user[i] = user[i].assign(dateTime = lambda x: (pd.to_datetime(user[i]['dateTime'], format="%Y-%m-%d %H:%M:%S")))

    data_windows = []
    for user in range(len(data)) :
        participant_windows = []
        for day in data[user]:
          half = day[day['dateTime'] >= pd.to_datetime(startTime, format="%Y-%m-%d %H:%M:%S")]
          final = half[half['dateTime'] <= pd.to_datetime(endTime, format="%Y-%m-%d %H:%M:%S")]
          participant_windows.append(final)
        data_windows.append(participant_windows)
    return data_windows

In [379]:

ph1 = PreProcess_Heart
pst2 = PreProcess_Steps
psl3 = PreProcess_Sleep
path = "20210602/"

In [380]:
ph1.MergeHeartData(path)
pst2.MergeStepData(path)
psl3.MergeSleepData(path)


In [381]:
ph1.ReOrder(path)
pst2.ReOrder(path)
psl3.ReOrder(path)

In [382]:
heart_Dfs = ph1.toDataFrame(path)
step_Dfs = pst2.toDataFrame(path)
sleep_Dfs = psl3.toDataFrame(path)

['merged/CR2002', 'merged/CR2003', 'merged/CR2005', 'merged/CR2006', 'merged/CR2007', 'merged/CR2010', 'merged/CR2012', 'merged/CR2014', 'merged/CR2016', 'merged/CR2017', 'merged/CR2018', 'merged/CR2019', 'merged/CR2020', 'merged/CR2021', 'merged/CR2022', 'merged/CR2023', 'merged/CR2024']


In [385]:
ph1.extractData(heart_Dfs, '2021-01-26 12:00:00', '2021-01-31 17:00:00')

[[Empty DataFrame
  Columns: [dateTime, value]
  Index: [],
                  dateTime  value
  717  2021-01-26 12:00:00     69
  718  2021-01-26 12:01:00     71
  719  2021-01-26 12:02:00     71
  720  2021-01-26 12:03:00     68
  721  2021-01-26 12:04:00     65
  ...                  ...    ...
  1397 2021-01-26 23:55:00     68
  1398 2021-01-26 23:56:00     68
  1399 2021-01-26 23:57:00     69
  1400 2021-01-26 23:58:00     72
  1401 2021-01-26 23:59:00     72
  
  [685 rows x 2 columns],
                  dateTime  value
  0    2021-01-27 00:00:00     72
  1    2021-01-27 00:01:00     70
  2    2021-01-27 00:02:00     70
  3    2021-01-27 00:03:00     70
  4    2021-01-27 00:04:00     70
  ...                  ...    ...
  1341 2021-01-27 23:55:00     85
  1342 2021-01-27 23:56:00     80
  1343 2021-01-27 23:57:00     82
  1344 2021-01-27 23:58:00     83
  1345 2021-01-27 23:59:00     83
  
  [1346 rows x 2 columns],
                  dateTime  value
  0    2021-01-28 00:00:00     

### Step Class

In [13]:
class Step_Features:

  def numOfSteps(step_data, window_size, value):
    new_data = PreProcess.extractData(step_data, window_size, value)
    columns = ['Total Steps', 'Mean', 'Median', 'St Dev']
    user_dfs = []
    for participant_data in range(len(new_data)):
      user_rows = []
      for day in range(len(new_data[participant_data])):
        for window in range(len(new_data[participant_data][day])):
            #print(new_data[0][0][0]['value'].values)
            total = sum(new_data[participant_data][day][window]['value'].values)
            avg = statistics.mean(new_data[participant_data][day][window]['value'].values)
            median = statistics.median(new_data[participant_data][day][window]['value'].values)
            stdev = statistics.stdev(new_data[participant_data][day][window]['value'].values)
            features = [total, avg, median, stdev]
            user_rows.append(features)
      user_dfs.append(pd.DataFrame(user_rows, columns=columns))
    return user_dfs   

  def activeEntropy(step_data, window_size, value):
    print('steps')
  
  def inactiveEntropy(step_data, window_size, value):
    print('steps')



In [14]:
test1 = Step_Features

In [15]:
step_results = test1.numOfSteps(step_dfs, 5, 0)


In [16]:
step_results[0]

,Total Steps,Mean,Median,St Dev
0,0,0,0.0,0.000000
1,2564,8,0.0,17.916473
2,6821,22,16.0,23.366643
3,3673,12,0.0,20.469489
4,807,3,0.0,9.695360
...,...,...,...,...
130,13,0,0.0,0.000000
131,1315,4,0.0,11.532563
132,2537,8,0.0,15.588457
133,1336,4,0.0,11.090537


### Heart Class

In [26]:
class Heart_Features:
  def heartCalc(heart_data, window_size, value):
    new_data = PreProcess.extractData(heart_data, window_size, value)
    columns = [ 'Mean', 'Median', 'St Dev']
    user_dfs = []
    for participant_data in range(len(new_data)):
      user_rows = []
      for day in range(len(new_data[participant_data])):
        for window in range(len(new_data[participant_data][day])):
            #print(new_data[0][0][0]['value'].values
            avg = statistics.mean(new_data[participant_data][day][window]['value'].values)
            median = statistics.median(new_data[participant_data][day][window]['value'].values)
            if len(new_data[participant_data][day][window]['value'].values) > 1:
                stdev = statistics.stdev(new_data[participant_data][day][window]['value'].values)
            else: 
                stdev = '-'
            features = [avg, median, stdev]
            user_rows.append(features)
      user_dfs.append(pd.DataFrame(user_rows, columns=columns))
    return user_dfs

In [28]:
heart_test = Heart_Features
heart_results = heart_test.heartCalc(heart_dfs, 5, 0)


In [31]:
len(heart_results[0])

135

In [37]:
heart_results[4]

,Mean,Median,St Dev
0,84,83.0,8.831761
1,82,81.0,6.480741
2,74,74.0,3.872983
3,68,70.0,2.236068
4,64,65.0,2.000000
5,77,73.0,13.453624
6,74,73.0,5.567764
7,80,79.0,7.141428
8,78,78.0,9.000000
9,71,72.0,2.449490


### Sleep Class

In [ ]:
class Sleep_Features:
  def heartCalc(heart_data, window_size, value):
    new_data = PreProcess.extractData(heart_data, window_size, value)
    columns = ['Duration', 'Sleep Regularity']
    user_dfs = []
    for participant_data in range(len(new_data)):
      user_rows = []
      for day in range(len(new_data[participant_data])):
        for window in range(len(new_data[participant_data][day])):
            #print(new_data[0][0][0]['value'].values
            avg = statistics.mean(new_data[participant_data][day][window]['value'].values)
            median = statistics.median(new_data[participant_data][day][window]['value'].values)
            if len(new_data[participant_data][day][window]['value'].values) > 1:
                stdev = statistics.stdev(new_data[participant_data][day][window]['value'].values)
            else: 
                stdev = '-'
            features = [avg, median, stdev]
            user_rows.append(features)
      user_dfs.append(pd.DataFrame(user_rows, columns=columns))
    return user_dfs